# Collecting, Cleaning and Uploading Data

In [65]:
import pandas as pd
import requests as r
import datetime
import sql_functions as sf
import psycopg2

### 1. UK bottled water Production (1998-2023) <br>

Output of mineral/bottled water by UK manufacturers in GBP Million.

[Source](https://www.ons.gov.uk/generator?format=csv&uri=/businessindustryandtrade/manufacturingandproductionindustry/timeseries/jq5i/diop)

In [66]:
production_columns = ['period', 'value']
production = pd.read_csv('data/bottled_water_production_UK.csv', 
                     names=production_columns, 
                     skiprows = 8)

In [67]:
production.head()

,period,value
0,1998,2820.2
1,1999,3187.3
2,2000,3057.8
3,2001,3145.7
4,2002,3240.8


In [68]:
#drop unnescessary rows by length and create datetime column
production_new = production[production['period'].str.len() == 8].copy()
production_new['date'] = pd.to_datetime(production_new['period'], format='%Y %b', dayfirst=True, errors='coerce')
production_new.head()

,period,value,date
128,1998 JAN,183.0,1998-01-01
129,1998 FEB,196.4,1998-02-01
130,1998 MAR,243.0,1998-03-01
131,1998 APR,247.7,1998-04-01
132,1998 MAY,287.9,1998-05-01


In [69]:
#rearrange and drop columns
uk_bottled_water_production = production_new.iloc[:,[2,1]]
uk_bottled_water_production.head()

,date,value
128,1998-01-01,183.0
129,1998-02-01,196.4
130,1998-03-01,243.0
131,1998-04-01,247.7
132,1998-05-01,287.9


In [70]:
uk_bottled_water_production.info()

<class 'pandas.core.frame.DataFrame'>
Index: 311 entries, 128 to 438
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    311 non-null    datetime64[ns]
 1   value   311 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 7.3 KB


In [71]:
# upload to DBeaver
#sf.upload(uk_bottled_water_production, 'uk_bottled_water_production')

### 2. UK bottled water  price (1996-2023) <br>
Output price inflation of mineral waters and other bottled waters. Base year for calculation is 2015 (=100) <br>
[Source](https://www.ons.gov.uk/generator?format=csv&uri=/businessindustryandtrade/manufacturingandproductionindustry/timeseries/jq5i/diop)

In [72]:
bw_price_columns = ['period', 'value']
bw_price = pd.read_csv('data/bottled_water_production_price_UK.csv', 
                     names=bw_price_columns, 
                     skiprows = 8)
bw_price.head()

,period,value
0,2009,95.6
1,2010,94.8
2,2011,94.8
3,2012,104.2
4,2013,107.5


In [73]:
#drop unnescessary rows by length and create datetime column
bw_price_new = bw_price[bw_price['period'].str.len() == 8].copy()
bw_price_new['date']=pd.to_datetime(bw_price_new['period'], format='%Y %b', dayfirst=True, errors='coerce')
bw_price_new.head()

,period,value,date
75,1996 JAN,77.0,1996-01-01
76,1996 FEB,80.6,1996-02-01
77,1996 MAR,78.6,1996-03-01
78,1996 APR,79.3,1996-04-01
79,1996 MAY,79.4,1996-05-01


In [74]:
uk_bottled_water_inflation_clean = bw_price_new.iloc[:,[2,1]]
uk_bottled_water_inflation_clean.head()

,date,value
75,1996-01-01,77.0
76,1996-02-01,80.6
77,1996-03-01,78.6
78,1996-04-01,79.3
79,1996-05-01,79.4


In [75]:
uk_bottled_water_inflation_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 336 entries, 75 to 410
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    336 non-null    datetime64[ns]
 1   value   336 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 7.9 KB


In [76]:
# upload to DBeaver
#sf.upload(uk_bottled_water_inflation_clean, 'uk_bottled_water_inflation_clean')

### 3. Top selling UK bottled water brands <br>
 Mineral contents, number of consumers and prices of most commonly bought brands in the UK.
 <br>
 top 10 list [Source(15.02.2023)](https://www.statista.com/statistics/308644/leading-brands-of-bottled-mineral-water-in-the-uk/) <br>
 nutritional values: tesco, other sources

In [77]:
bottled_water_nutritions = pd.read_csv('data/Bottled_water_references - Sheet1.csv', )
bottled_water_nutritions.head()

,gibrand,owner,consumers,type,price_per_liter_gbp,sulfate_so42,bicarbonate_hco,calcium_ca++,chloride_cl-,total_dissolved_solids_at_180c,magnesium_mg2,sodium_na+,silica_sio2,nitrate_no3-,potassium_k+,fluoride_f-,ph,remark
0,Highland Spring Water,Highland Spring,6132550,still,0.6,5.3,150.0,40.5,6.1,170.0,10.1,5.6,NaN,3.1,0.7,NaN,NaN,NaN
1,Evian,Danone,5939000,still,1.0,14.0,360.0,80.0,10.0,345.0,26.0,6.5,15.0,3.8,1.0,NaN,NaN,NaN
2,Buxton Spring,Nestle,5141140,still,0.6,13.0,248.0,55.0,37.0,280.0,19.0,24.0,NaN,0.1,1.0,NaN,7.4,NaN
3,Volvic,Danone,4489750,still,0.7,9.0,74.0,12.0,15.0,130.0,8.0,12.0,32.0,7.3,6.0,NaN,NaN,NaN
4,Tesco,Tesco,3748080,still,0.2,11.0,25.0,11.0,14.0,NaN,3.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
bottled_water_nutritions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   gibrand                         18 non-null     object 
 1   owner                           18 non-null     object 
 2   consumers                       18 non-null     int64  
 3   type                            18 non-null     object 
 4   price_per_liter_gbp             18 non-null     float64
 5   sulfate_so42                    18 non-null     float64
 6   bicarbonate_hco                 18 non-null     float64
 7   calcium_ca++                    18 non-null     float64
 8   chloride_cl-                    17 non-null     float64
 9   total_dissolved_solids_at_180c  15 non-null     float64
 10  magnesium_mg2                   18 non-null     float64
 11  sodium_na+                      17 non-null     float64
 12  silica_sio2                     4 non-

In [79]:
# upload to DBeaver
#sf.upload(bottled_water_nutritions, 'bottled_water_nutritions')

### 4. UK water supply price (1988-2023)
[Source](https://www.ons.gov.uk/generator?format=csv&uri=/economy/inflationandpriceindices/timeseries/d7dr/mm23)

In [80]:
#read csv file
water_supply_columns = ['date', 'value']
water_supply = pd.read_csv('data/water_supply_price_UK.csv',
                     names=water_supply_columns, 
                     skiprows = 188) # first rows have accumulated years and quarters
water_supply.head()

,date,value
0,1988 JAN,21.2
1,1988 FEB,21.2
2,1988 MAR,21.2
3,1988 APR,22.5
4,1988 MAY,22.5


In [81]:
# change the column 'period' into datetime format
water_supply['date'] = pd.to_datetime(water_supply['date'], dayfirst = True)
water_supply.head(2)

/var/folders/9j/5rxmqbbd545bhw9vljt6mdp40000gn/T/ipykernel_53574/3975858738.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  water_supply['date'] = pd.to_datetime(water_supply['date'], dayfirst = True)


,date,value
0,1988-01-01,21.2
1,1988-02-01,21.2


In [82]:
water_supply.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    432 non-null    datetime64[ns]
 1   value   432 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 6.9 KB


In [83]:
# upload to DBeaver
# sf.upload(water_supply,'water_supply_price')

### 5. Anglian Water Domestic Water Qualitiy (2022) <br>
Results of all water tests in Anglian Water suplly region <br>
[Source](https://streamwaterdata.co.uk/details/21)<br>
[Drinking Water Standards](https://www.dwi.gov.uk/drinking-water-standards-and-regulations/)

In [84]:
aw_quality = pd.read_csv('data/anglian_water_quality_2022.csv')
aw_quality.head(2)

,Unnamed: 0,Sample_Id,Sample_Date,DWI_Code,Determinand,Units,Operator,Result,LSOA21CD,Data_Provider
0,0,0d8ac3ad-a3f4-49a4-8810-261d0a9d9046,03/01/2022 00:00,C001A,Coliform Bacteria (Indicator),No. /100ml,NaN,0.0,E01021706,Anglian Water
1,1,0d8ac3ad-a3f4-49a4-8810-261d0a9d9046,03/01/2022 00:00,C002,E.Coli (faecal coliforms Confirmed),No. /100ml,NaN,0.0,E01021706,Anglian Water


In [85]:
#drop columns
aw_quality.drop(['Unnamed: 0','Data_Provider','Operator'], axis = 1, inplace=True)

In [86]:
#naming convention
aw_quality.columns = aw_quality.columns.str.lower()
aw_quality.head(2)

,sample_id,sample_date,dwi_code,determinand,units,result,lsoa21cd
0,0d8ac3ad-a3f4-49a4-8810-261d0a9d9046,03/01/2022 00:00,C001A,Coliform Bacteria (Indicator),No. /100ml,0.0,E01021706
1,0d8ac3ad-a3f4-49a4-8810-261d0a9d9046,03/01/2022 00:00,C002,E.Coli (faecal coliforms Confirmed),No. /100ml,0.0,E01021706


In [87]:
aw_quality.sample_date = pd.to_datetime(aw_quality.sample_date, format= '%d/%m/%Y %H:%M')

In [88]:
aw_quality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130664 entries, 0 to 130663
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   sample_id    130664 non-null  object        
 1   sample_date  130664 non-null  datetime64[ns]
 2   dwi_code     130664 non-null  object        
 3   determinand  130447 non-null  object        
 4   units        130664 non-null  object        
 5   result       130664 non-null  float64       
 6   lsoa21cd     130664 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 7.0+ MB


In [89]:
#upload to DBeaver
#sf.upload(aw_quality, 'water_quality')

### 6. Weather Data UK annualy (1884-2024) <br>
average temperature per month <br>
[Source](https://www.metoffice.gov.uk/research/climate/maps-and-data/uk-and-regional-series) 


In [90]:
weather = pd.read_csv('https://www.metoffice.gov.uk/pub/data/weather/uk/climate/datasets/Tmean/date/UK.txt',sep='   ' ,skiprows=5, engine='python')
weather.head(2)

,year,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,win,spr,sum,aut,ann
0,1884,5.0,4.2,5.1,6.2,9.7,12.5,14.5,15.3,13.0,8.3,4.6,3.3,---,6.98,14.11,8.62,8.48
1,1885,2.0,4.2,3.5,6.6,7.6,12.3,14.5,12.4,10.9,6.2,4.8,3.0,3.14,5.92,13.09,7.28,7.36


In [91]:
weather.columns

Index(['year', ' jan', ' feb', ' mar', ' apr', ' may', ' jun', ' jul', ' aug',
       ' sep', ' oct', ' nov', ' dec', '  win', '  spr', '  sum', '  aut',
       '  ann'],
      dtype='object')

In [92]:
# strip the spaces from header
weather.columns = weather.columns.str.lstrip()

In [93]:
#drop seasons and ann
weather.drop(['win', 'spr', 'sum', 'aut','ann'], axis=1 , inplace =True)

In [94]:
#drop unnecessary years
weather.drop(range(104),axis=0 ,inplace =True)

In [95]:
#update row index
weather.index = range(len(weather))
weather.head()

,year,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
0,1988,4.1,3.9,5.1,7.1,10.5,13.3,13.5,13.9,12.1,9.4,5.0,6.6
1,1989,5.8,4.9,6.0,5.5,11.3,12.9,16.4,14.8,12.9,10.5,5.7,3.6
2,1990,5.5,5.9,7.2,7.0,11.2,12.4,15.2,16.1,11.7,10.5,5.8,3.7
3,1991,2.5,1.3,6.6,7.0,9.9,11.0,15.9,15.6,13.1,9.0,5.7,4.2
4,1992,3.4,4.8,6.1,7.4,11.9,14.3,14.7,13.8,11.9,6.7,6.1,3.1


In [96]:
weather.columns

Index(['year', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep',
       'oct', 'nov', 'dec'],
      dtype='object')

In [97]:
weather.rename(columns ={'jan':'1', 'feb':'2', 'mar':'3', 'apr':'4', 'may':'5', 'jun':'6', 'jul':'7', 'aug':'8', 'sep':'9',
       'oct':'10', 'nov':'11', 'dec':'12'}, inplace = True)
weather.head(2)

,year,1,2,3,4,5,6,7,8,9,10,11,12
0,1988,4.1,3.9,5.1,7.1,10.5,13.3,13.5,13.9,12.1,9.4,5.0,6.6
1,1989,5.8,4.9,6.0,5.5,11.3,12.9,16.4,14.8,12.9,10.5,5.7,3.6


In [98]:
#create new table with better suited structure
y = 0
m = 1
weather_new = pd.DataFrame({'year':[],'month':[],'temp':[]})

while y < len(weather.index):
        while m <= 12: 
                weather_new.loc[len(weather_new.index)] = [weather.iloc[y,0],m, weather.iloc[y,m]]
                m = m+1
        m=1
        y=y+1

In [99]:
#define datatype of columns
weather_new = weather_new.astype({'year':'int', 'month':'int'})
weather_new['date'] = weather_new.year.astype(str)+'-' + weather_new.month.astype(str) +'-' + '1'
weather_new.date = pd.to_datetime(weather_new.date)

In [100]:
#drop columns
weather_new.drop(['year','month'], axis = 1, inplace= True)
weather_new.head()

,temp,date
0,4.1,1988-01-01
1,3.9,1988-02-01
2,5.1,1988-03-01
3,7.1,1988-04-01
4,10.5,1988-05-01


In [101]:
weather_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 444 entries, 0 to 443
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   temp    433 non-null    float64       
 1   date    444 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1)
memory usage: 10.4 KB


In [102]:
# upload to DBeaver
#sf.upload(weather_new, 'weather_uk')

### 7. Trust in Government G7 (2006-2021) <br>
[Source](https://data.oecd.org/gga/trust-in-government.htm) <br>
OECD (2024), Trust in government (indicator). doi: 10.1787/1de9675e-en (Accessed on 12 February 2024)

In [103]:
trust = pd.read_csv('data/Trust_in_Gov_UK.csv',)
trust.head(2)

,Location,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Canada,44.3,63.8,59.0,60.8,55.1,55.3,52.3,50.6,51.7,64.4,61.8,65.3,61.0,54.9,60.0,61.0,50.7
1,France,32.5,36.1,45.3,47.4,40.1,37.5,44.1,39.5,26.4,32.8,28.4,37.5,38.1,38.2,41.0,43.4,NaN


In [104]:
# transpose dataframe
trust.set_index('Location', inplace = True)
trustT = trust.T.copy()
trustT.reset_index(names='year',inplace=True) #new index + column with old index values named'year'
trustT= trustT.rename_axis(None, axis=1) #deletes the axis name 'location'
trustT.head(2)

,year,Canada,France,Germany,Italy,Japan,United Kingdom,United States
0,2006,44.3,32.5,32.2,23.9,35.0,49.5,55.8
1,2007,63.8,36.1,35.0,30.3,24.0,36.4,39.3


In [105]:
trustT.columns= trustT.columns.str.lower()
trustT.year = trustT.year.astype(int)
trustT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            17 non-null     int64  
 1   canada          17 non-null     float64
 2   france          16 non-null     float64
 3   germany         17 non-null     float64
 4   italy           16 non-null     float64
 5   japan           17 non-null     float64
 6   united kingdom  16 non-null     float64
 7   united states   17 non-null     float64
dtypes: float64(7), int64(1)
memory usage: 1.2 KB


In [106]:
# upload to DBeaver
# sf.upload(trustT, 'trust_in_gov')

### 8. UK Disposable Income per Capita (1995-2022)
[Source](https://www.ons.gov.uk/economy/grossdomesticproductgdp/timeseries/mwb7/ukea)

In [107]:
income_columns = ['quarter', 'value']
income = pd.read_csv('data/UK_income_capita.csv', 
                     names=income_columns, 
                     skiprows = 36)
income.head(2)

,quarter,value
0,1995 Q1,4440
1,1995 Q2,4444


In [108]:
income.quarter= income.quarter.str.replace(' ','-')
income.quarter = pd.to_datetime(income.quarter, format = 'mixed')
income.head()

,quarter,value
0,1995-01-01,4440
1,1995-04-01,4444
2,1995-07-01,4501
3,1995-10-01,4553
4,1996-01-01,4588


In [109]:
#upload to DBeaver
#sf.upload(income,'income_uk')

### 9. Socio-Political Events <br>
a list of 'random' events to checl weather they have an impact

In [110]:
events = pd.read_csv('data/events.csv')
events.head(2)

,event,start_date,end_date
0,Foot-and-mouth disease outbreak,01.02.01,30.09.01
1,World Trade Center attacks,11.09.01,NaN


In [111]:
events['start_date']= pd.to_datetime(events['start_date'], dayfirst = True, format = 'mixed')
events['end_date']= pd.to_datetime(events['end_date'], dayfirst = True, format = 'mixed')
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   event       14 non-null     object        
 1   start_date  14 non-null     datetime64[ns]
 2   end_date    4 non-null      datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 464.0+ bytes


In [112]:
# upload to DBeaver
#sf.upload(events, 'events')

## 10. LSOA 21CD and postal codes UK<br>
[Source](https://geoportal.statistics.gov.uk/datasets/ons::postcode-to-2021-census-output-area-to-lower-layer-super-output-area-to-middle-layer-super-output-area-to-local-authority-district-may-2023-lookup-in-the-uk/about) 

In [113]:
post = pd.read_csv('data/Postcode_LSOA21.csv',encoding='latin-1',low_memory=False)
post.head()

,pcd7,pcd8,pcds,dointr,doterm,usertype,oa21cd,lsoa21cd,msoa21cd,ladcd,lsoa21nm,msoa21nm,ladnm,ladnmw
0,AB1 0AA,AB1 0AA,AB1 0AA,198001,199606.0,0,S00090303,S01006514,S02001237,S12000033,NaN,NaN,Aberdeen City,NaN
1,AB1 0AB,AB1 0AB,AB1 0AB,198001,199606.0,0,S00090303,S01006514,S02001237,S12000033,NaN,NaN,Aberdeen City,NaN
2,AB1 0AD,AB1 0AD,AB1 0AD,198001,199606.0,0,S00090399,S01006514,S02001237,S12000033,NaN,NaN,Aberdeen City,NaN
3,AB1 0AE,AB1 0AE,AB1 0AE,199402,199606.0,0,S00091322,S01006853,S02001296,S12000034,NaN,NaN,Aberdeenshire,NaN
4,AB1 0AF,AB1 0AF,AB1 0AF,199012,199207.0,1,S00090299,S01006511,S02001236,S12000033,NaN,NaN,Aberdeen City,NaN


In [114]:
post_21 = post[['pcd7','lsoa21cd','ladnm']].copy()
post_21.head()

,pcd7,lsoa21cd,ladnm
0,AB1 0AA,S01006514,Aberdeen City
1,AB1 0AB,S01006514,Aberdeen City
2,AB1 0AD,S01006514,Aberdeen City
3,AB1 0AE,S01006853,Aberdeenshire
4,AB1 0AF,S01006511,Aberdeen City


In [115]:
post_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2690851 entries, 0 to 2690850
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   pcd7      object
 1   lsoa21cd  object
 2   ladnm     object
dtypes: object(3)
memory usage: 61.6+ MB


In [116]:
#
post_21.lsoa21cd.nunique()

43500

In [131]:
aw_quality.lsoa21cd.nunique() #for later confirmation

2933

In [127]:
#list of unique areas
codes = aw_quality.lsoa21cd.unique()

In [128]:
#create subset
post_21_aw = post_21[post_21.lsoa21cd.isin(codes)]


In [130]:
post_21_aw.lsoa21cd.nunique() #confirmation

2933

In [137]:
post_21_aw.index = range(len(post_21_aw))
post_21_aw.rename(columns={'pcd7':'postcode','ladnm':'city'}, inplace=True)
post_21_aw.head()

/var/folders/9j/5rxmqbbd545bhw9vljt6mdp40000gn/T/ipykernel_53574/1665786288.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_21_aw.rename(columns={'pcd7':'postcode','ladnm':'city'}, inplace=True)


,postcode,lsoa21cd,city
0,B49 5JH,E01032392,Wychavon
1,B49 5JR,E01032392,Wychavon
2,B49 5JS,E01032392,Wychavon
3,B49 5JT,E01032392,Wychavon
4,B49 5JU,E01032392,Wychavon


In [138]:
#upload to DBeaver
#sf.upload(post_21_aw,'postcode_lsoa_aw')

The postcode_lsoa_aw table was imported successfully.
